In [66]:
from datascience import *
import numpy as np
import re
from re import search
import requests
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

In [67]:
# scrape_table function from video demo
import pandas as pd
def scrape_table(table):
    df = pd.read_html(str(table))
    df = pd.DataFrame(df[0])
    return Table.from_df(df)

In [68]:
r = requests.get("https://www.numbeo.com/cost-of-living/prices_by_country.jsp?itemId=6&regionCode=019&displayCurrency=CAD")
coke_text = BeautifulSoup(r.text, "html.parser")
coke_text_table = coke_text.find_all("table")[1]
coke_table = scrape_table(coke_text_table).drop("Rank")
coke_table

Country,Coke/Pepsi (0.33 liter bottle)
Kazakhstan,0.78
Paraguay,1.18
Portugal,1.96
Greece,2.53
Latvia,1.56
Iran,1.16
Morocco,0.63
Panama,2.04
Guatemala,1.38
Iraq,0.58


In [69]:
def convert(cur_from, cur_to, amount):
    payload = {
        'from': cur_from,
        'to': cur_to,
        'amount': amount,
        'api_key': '0a5659758c74b6dd01f96fdbc8fab87d'
    }
    r = requests.get('https://api.currencyscoop.com/v1/convert', params = payload)
    currency = r.json()
    return currency['response']['value']

In [70]:
#stolen shamlessly from example code
currency_url = "https://currencyscoop.com/supported-currencies"
currency_request = requests.get(currency_url)
currency_soup = BeautifulSoup(currency_request.text, 'html.parser')
currency_table = currency_soup.find("table")
currency_codes = scrape_table(currency_table)

codes = make_array()
for each_country in currency_codes.column("Currency"):
    codes = np.append(codes, each_country[-3:])

currency_codes = currency_codes.with_column("Code", codes)

def strip_code(country):
    return country[:-3]

currency_codes["Currency"] = currency_codes.apply(strip_code, "Currency")

In [71]:
import pycountry

mapping = {country.name: country.alpha_2 for country in pycountry.countries}
coke_table = coke_table.with_column("Country Code", coke_table.apply(lambda x: mapping.get(x), "Country"))

In [72]:
r = requests.get("https://www.iban.com/currency-codes")
currency_code_table = scrape_table(BeautifulSoup(r.text, "html.parser").find_all("table")[0])
type(currency_code_table.column(2)[0])

numpy.str_

In [73]:
def get_currency_code(country):
    i = np.where(currency_code_table.column("Country") == country.upper())
    return currency_code_table.column("Code")[i]

def clean_currency_code(code):
    cleaned_code = np.array_str(code)[2:5]
    return cleaned_code

coke_table = coke_table.with_column("Currency Code", coke_table.apply(get_currency_code, "Country"))
coke_table = coke_table.with_column("Currency Code", coke_table.apply(clean_currency_code, "Currency Code"))

coke_table

/Users/tpurewal/opt/anaconda3/lib/python3.9/site-packages/datascience/tables.py:558: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([fn(*row) for row in rows])


Country,Coke/Pepsi (0.33 liter bottle),Country Code,Currency Code
Kazakhstan,0.78,KZ,KZT
Paraguay,1.18,PY,PYG
Portugal,1.96,PT,EUR
Greece,2.53,GR,EUR
Latvia,1.56,LV,EUR
Iran,1.16,None,
Morocco,0.63,MA,MAD
Panama,2.04,PA,PAB
Guatemala,1.38,GT,GTQ
Iraq,0.58,IQ,IQD


In [74]:
# tidy up currency symbol and GST rate
currencies = make_array()
gst_rates = make_array()

# loop through each country
for each_country in coke_table.column("Country"):
    # grab currency symbol only
  

    # grab GST rate only -- some countries don't have one, so I used regex to find perfect match
    match = search("\d\d%", each_country)
    if match != None:
        gst_rates = np.append(gst_rates, match.group(0))

    else:
        gst_rates = np.append(gst_rates, '0%')

# tack on new columns
coke_table = coke_table.with_column("GST Rate", gst_rates)
coke_table

Country,Coke/Pepsi (0.33 liter bottle),Country Code,Currency Code,GST Rate
Kazakhstan,0.78,KZ,KZT,0%
Paraguay,1.18,PY,PYG,0%
Portugal,1.96,PT,EUR,0%
Greece,2.53,GR,EUR,0%
Latvia,1.56,LV,EUR,0%
Iran,1.16,None,,0%
Morocco,0.63,MA,MAD,0%
Panama,2.04,PA,PAB,0%
Guatemala,1.38,GT,GTQ,0%
Iraq,0.58,IQ,IQD,0%
